In [14]:
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import feature_selection
import seaborn as sns
import openpyxl

In [30]:
janela = 'mensal'
book_name='book_pagaya_riverview'
df = pd.read_csv("P:\\ciencia_de_dados\\Correlacao_de_fundos\\Bases\\"  + book_name + "_" + janela + ".csv",delimiter=',',decimal='.',parse_dates=True)
df = df.dropna().drop_duplicates().rename(columns={'Unnamed: 0':'data'})
df.head()

,data,Pagaya Opportunity Fund,Riverview ALF
2,2018-09-28,105.47449304121544,100.0
3,2018-10-31,106.17062469528746,100.12
4,2018-11-30,106.9987555679107,100.73
5,2018-12-31,107.66214785243174,101.28999999999999
6,2019-01-31,108.42654910218403,101.83


In [31]:
df = df.melt(id_vars='data', value_vars=list(df.columns[1:]), value_name="FinancialPrice",var_name='Product')

In [32]:
#Calculo das colunas temporais
df['data']=pd.to_datetime(df['data'],format = '%Y-%m-%d')
df['MesAno'] = df['data'].dt.strftime('%m-%Y')
df['Mes_nomial']=df['data'].dt.strftime('%b')
df['Mes']=df['data'].dt.strftime('%m').astype(int)
df['Ano']=df['data'].dt.strftime('%Y').astype(int)

In [33]:
if janela == 'diaria':
    df = df.groupby('MesAno').agg(['max'])
    df.columns = df.columns.droplevel(1)
    df = df.reset_index()

In [34]:
df.sort_values(by=["Product",'data'],ascending=False,inplace=True)

In [35]:
df

,data,Product,FinancialPrice,MesAno,Mes_nomial,Mes,Ano
67,2021-06-30,Riverview ALF,151.3774543057501,06-2021,Jun,6,2021
66,2021-05-31,Riverview ALF,148.31780627999999,05-2021,May,5,2021
65,2021-04-30,Riverview ALF,145.32,04-2021,Apr,4,2021
64,2021-03-31,Riverview ALF,145.22,03-2021,Mar,3,2021
63,2021-02-26,Riverview ALF,133.53,02-2021,Feb,2,2021
...,...,...,...,...,...,...,...
4,2019-01-31,Pagaya Opportunity Fund,108.42654910218403,01-2019,Jan,1,2019
3,2018-12-31,Pagaya Opportunity Fund,107.66214785243174,12-2018,Dec,12,2018
2,2018-11-30,Pagaya Opportunity Fund,106.9987555679107,11-2018,Nov,11,2018
1,2018-10-31,Pagaya Opportunity Fund,106.17062469528746,10-2018,Oct,10,2018


In [36]:
#Cria a coluna retorno e seta NaN na primeira linha de cada fundo
df['FinancialPrice']=pd.to_numeric(df["FinancialPrice"], downcast="float")
df["Retorno_1"] = (df.FinancialPrice.shift())/(df.FinancialPrice)
df.loc[df.drop_duplicates('Product').index.values,'Retorno_1'] = np.NaN
df['Retorno'] = df.Retorno_1.shift(periods=-1)
df['Retorno'] = (df["Retorno_1"]-1)

In [37]:
df.to_csv('P:\\ciencia_de_dados\\Correlacao_de_fundos\\Desenvolvimento\\1DataPrep\\base_'+ book_name + '.csv',sep =';',decimal=',')